In [ ]:
from tastytrade.logging import setup_logging
import logging
import asyncio
from tastytrade.sessions import Credentials

from tastytrade.sessions.enumerations import Channels
from tastytrade.sessions.sockets import DXLinkManager

from tastytrade.visualization.candlesticks import plot_live_candlesticks

from datetime import datetime, timedelta

logging.getLogger().handlers.clear()

TEST = True
ENV = "Live"
DURATION = 15

start_time = datetime.now() - timedelta(minutes=30)
end_time = datetime.now() - timedelta(minutes=2)

setup_logging(
    level=logging.INFO,
    log_dir="../logs",
    filename_prefix=f"{'dev' if TEST else 'prod'}_tastytrade",
    console=True,
    file=True,
)

loop = asyncio.get_event_loop()
loop.set_debug(True)
logging.getLogger("asyncio").setLevel(logging.DEBUG)

## Test individual components

In [ ]:
# Set API credentials
credentials = Credentials(env=ENV)


credentials = Credentials(env="Live")
dxlink = DXLinkManager()

await dxlink.open(credentials)

In [3]:
await dxlink.subscribe_to_candles(
    symbol="BTC/USD:CXTALP",
    # symbol="SPY",
    interval="1m",
    # from_time=datetime.now(),
    from_time=start_time,
    # to_time=end_time,
)

In [4]:
# Subscribe to 5-minute candles for SPY
# await dxlink.unsubscribe_to_candles(symbol="BTC/USD:CXTALP", interval="1m")

In [5]:
symbols = ["BTC/USD:CXTALP", "SPY", "NVDA"]
await dxlink.subscribe(symbols)

In [6]:
# await dxlink.unsubscribe(symbols)

In [ ]:
# Access candle data
columns = ["eventSymbol", "time", "open", "high", "low", "close"]
dxlink.router.handler[Channels.Candle].processors["feed"].df[columns].tail(10)

In [ ]:
dxlink.router.handler[Channels.Trade].processors["feed"].df.tail(10)

In [ ]:
dxlink.router.handler[Channels.Quote].processors["feed"].df.tail(10)

In [ ]:
dxlink.router.handler[Channels.Greeks].processors["feed"].df

In [ ]:
dxlink.router.handler[Channels.Profile].processors["feed"].df

In [ ]:
dxlink.router.handler[Channels.Summary].processors["feed"].df

In [ ]:
task = plot_live_candlesticks(dxlink, "BTC/USD:CXTALP{=m}")

In [ ]:
task.cancel()

In [ ]:
await dxlink.close()